In [1]:
import fasttext
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Lambda

# Have to download the stopwords
# nltk.download('stopwords')

In [2]:
# Get the fasttext model (we are using the largest one they offer [600B tokens])
fasttext_model = fasttext.load_model('models/crawl-300d-2M-subword.bin')

In [ ]:
# Creates the stopwords
to_stop = stopwords.words('english')
punctuation = "!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~ "
for c in punctuation:
    to_stop.append(c)

to_stop.append('null')
print(to_stop)

In [ ]:
# Organizing and normalizing the data
"""
Essentially, we want to only have three attributes for each training example: title_one, title_two, label
For normalization, we are just going to use the nltk stopwords and punctuation
"""

def preprocessing(orig_data):
    """
    Normalizes the data by getting rid of stopwords and punctuation
    """
    
    # The new names of the columns
    column_names = ['title_one', 'title_two', 'label']
    # A new dataframe for the data we are going to be creating
    norm_computers = pd.DataFrame(columns = column_names)
    # Iterate over the original dataframe (I know it is slow and there are probably better ways to do it)
    for row in orig_data.itertuples():
        title_left = row.title_left.split(' ')
        title_right = row.title_right.split(' ')
        
        # Creates a new list of only elements that are not in the stop words
        temp_title_left = []
        for word in title_left:
            if word not in to_stop:
                temp_title_left.append(word)
                
        # Creates a new list of only elements that are not in the stop words
        temp_title_right = []
        for word in title_right:
            if word not in to_stop:
                temp_title_right.append(word)
        
        # Join the elements in the list to create the strings
        title_left = ' '.join(temp_title_left)
        title_right = ' '.join(temp_title_right)
        
        # Append the newly created row (title_left, title_right, label) to the new dataframe
        norm_computers = norm_computers.append(pd.DataFrame([[title_left, title_right, row.label]], columns=column_names))
        
    return norm_computers
        

In [ ]:
def create_simple_data():
    """
    Creates and saves a simpler version of the original data that only contains the the two titles and the label.
    """
    
    # Get the dataset of computer parts
    computers_df = pd.read_json('data/computers_train/computers_train_xlarge_normalized.json.gz',compression='gzip', lines=True)
    norm_computers = preprocessing(computers_df)
    
    # Save the new normalized and simplified data to a CSV file to load later
    norm_computers.to_csv('data/computers_train/computers_train_xlarge_norm_simple.csv', index=False)

In [ ]:
# Load the data
computer_df = pd.read_csv('data/computers_train/computers_train_xlarge_norm_simple.csv')

In [ ]:
computer_df.head()

## Model Info

For the model, we are going to use LSTMs with a Constrastive Loss Function 
that will also be used to predict whether the two products are the same 

First, we have to convert the titles to embeddings through FastText before feeding into the LSTM.
The embedding part of this model will not be a layer because:
* The fasttext model would be time consuming and annoying to get to work with an embedding layer in Keras
* The fasttext model is not going to be getting its embeddings optimized, so there is really no point in adding it as an embedding layer

In [22]:
def euclidean_distance(vectors):
    x, y = vectors
    return tf.reduce_sum(tf.square(x - y), 1)

def euclidean_dist_out_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def siamese_network(input_shape):
    # Defines our inputs
    left_title = Input(input_shape, dtype='float32')
    right_title = Input(input_shape, dtype='float32')
    
    # The LSTM units
    model = tf.keras.Sequential(name='siamese_model')
    model.add(LSTM(units=256, return_sequences=True, name='lstm_1'))
    model.add(Dropout(rate=0.3))
    model.add(LSTM(units=128, return_sequences=True, name='lstm_2'))
    model.add(Dropout(rate=0.3))
    model.add(LSTM(units=128, name='lstm_3'))
    model.add(Dropout(rate=0.3))
    
    # The dense layers
    model.add(Dense(units=1024, activation='relu', name='dense_1'))
    model.add(Dropout(rate=0.3))
    model.add(Dense(units=512, activation='relu', name='dense_2'))
    
    # Forward propagate through the model to generate the encodings
    encoded_left_title = model(left_title)
    encoded_right_title = model(right_title)
    print(encoded_left_title)
    distance = Lambda(euclidean_distance, output_shape=euclidean_dist_out_shape)([encoded_left_title, encoded_right_title])

    # Create and return the network
    siamese_net = tf.keras.Model(inputs=(left_title, right_title), outputs=distance, name='siamese_network')
    return siamese_net

In [4]:
def get_max_len():
    max_len = 0
    for row in computer_df.itertuples():
        if len(row.title_one.split(' ')) > max_len:
            max_len = len(row.title_one.split(' '))
            
        if len(row.title_two.split(' ')) > max_len:
            max_len = len(row.title_two.split(' '))
    
    return max_len


In [3]:
"""
Create the numpy files of all the training embedddings
We will have two numpy files:
1. The training/validation/test sets
2. The labels
"""

def create_embeddings(df):
    # Create the numpy arrays for storing the embeddings and labels
    total_embeddings = np.zeros(shape=(m, 2, MAX_LEN, EMBEDDING_SHAPE[0]))
    labels = np.zeros(shape=(m))
    
    # I know this is a terrible way of doing this, but iterate over the dataframe
    # and generate the embeddings to add to the numpy array
    for idx, row in enumerate(computer_df.itertuples()):
        for word_idx, word in enumerate(row.title_one.split(' ')):
            total_embeddings[idx, 0, word_idx] = fasttext_model[word]
            
        for word_idx, word in enumerate(row.title_two.split(' ')):
            total_embeddings[idx, 1, word_idx] = fasttext_model[word]
            
        labels[idx] = row.label
        
    return total_embeddings, labels


In [4]:
def save_embeddings():
    embeddings, labels = create_embeddings(computer_df)
    with open('data/computers_numpy/embeddings.npy', 'wb') as f:
        np.save(f, embeddings)

    with open('data/computers_numpy/labels.npy', 'wb') as f:
        np.save(f, labels)

In [5]:
def load_embeddings_and_labels():
    loaded_embeddings = None
    labels = None
    with open('data/computers_numpy/embeddings.npy', 'rb') as f:
        loaded_embeddings = np.transpose(np.load(f), (1, 0, 2, 3))
    
    with open('data/computers_numpy/labels.npy', 'rb') as f:
        labels = np.load(f)
    
    return loaded_embeddings, labels

In [6]:
embeddings, labels = load_embeddings_and_labels()
print('Embeddings shape: ' + str(embeddings.shape), 'Labels shape: ' + str(labels.shape))

Embeddings shape: (2, 68461, 42, 300) Labels shape: (68461,)


In [7]:
"""
Definitions of some sizes in the training set
"""
# Returns the max word length in the entire training set
margin = 2.0
MAX_LEN = 42
#EMBEDDING_SHAPE = fasttext_model['queen'].shape
EMBEDDING_SHAPE = (300,)
m = embeddings.shape[1]
print('MAX_LEN: ' + str(MAX_LEN), 'EMBEDDING_SHAPE: ' + str(EMBEDDING_SHAPE), 'm: ' + str(m))

MAX_LEN: 42 EMBEDDING_SHAPE: (300,) m: 68461


In [8]:
# Note: for the constrastive loss, because 0 denotes that they are from the same class
# and one denotes they are from a different class, I swaped the (Y) and (1 - Y) terms

# Just chose this
def constrastive_loss(y_true, y_pred):
    d = y_pred
    d_sqrt = tf.sqrt(d)

    loss = y_true * d + (y_true - 1) * tf.square(tf.maximum(0., margin - d_sqrt)) 

    loss = 0.5 * tf.reduce_mean(loss)
    
    return loss

In [9]:
# Previous testing code
#constrastive_loss(np.array([[2.0, 3.0, 4.0], [3.0, 4.0, 5.0]], dtype=np.float32), np.array([[3, 4, 5], [4, 5, 6]], dtype=np.float32), 0)

In [10]:
# Accuracy metric for constrastive loss because values close to 0 are equal and values high are different
# 0.5 is the threshold here
def constrastive_accuracy(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(y_true, tf.cast(y_pred < 0.5, y_true.dtype)), y_true.dtype))

In [ ]:
constrastive_accuracy(np.array([1, 1, 0], dtype=np.float32), np.array([0.3, 1.324, 0.9], dtype=np.float32))

In [11]:
X_train1 = embeddings[0, :50000]
X_train2 = embeddings[1, :50000]
X_train = np.stack((X_train1, X_train2))
print('Training shape: ' + str(X_train.shape))


X_val1 = embeddings[0, 50000:59000]
X_val2 = embeddings[1, 50000:59000]
X_val = np.stack((X_val1, X_val2))
print('Val shape: ' + str(X_val.shape))

X_test1 = embeddings[0, 59000:]
X_test2 = embeddings[1, 59000:]
X_test = np.stack((X_test1, X_test2))
print('Test shape: ' + str(X_test.shape))

Training shape: (2, 50000, 42, 300)
Val shape: (2, 9000, 42, 300)
Test shape: (2, 9461, 42, 300)


In [12]:
del embeddings

In [13]:
Y_train = labels[:50000]
Y_val = labels[50000:59000]
Y_test = labels[59000:]

In [14]:
del labels

In [23]:
model = siamese_network((MAX_LEN, EMBEDDING_SHAPE[0],))
model.summary()

Tensor("siamese_model_4/Identity:0", shape=(None, 512), dtype=float32)
Model: "siamese_network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 42, 300)]    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 42, 300)]    0                                            
__________________________________________________________________________________________________
siamese_model (Sequential)      (None, 512)          1555968     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lam

In [27]:
model.compile(loss=constrastive_loss, optimizer='adam', metrics=['accuracy', constrastive_accuracy])
model.fit(x=[X_train1, X_train2], y=Y_train, batch_size=64, epochs=2, validation_data=([X_val[0], X_val[1]], Y_val))

Tensor("lambda_2_target_2:0", shape=(None,), dtype=float32) Tensor("lambda_2/Identity:0", shape=(None,), dtype=float32)


MemoryError: Unable to allocate 4.69 GiB for an array with shape (50000, 42, 300) and data type float64

In [21]:
results = model.evaluate([X_test1, X_test2], Y_test, batch_size=64)
print('test loss, test acc: ', results)

9461/9461 [==============================] - 26s 3ms/sample - loss: nan - accuracy: 1.0000 - constrastive_accuracy: 1.0000
test loss, test acc:  [nan, 1.0, 1.0]


NameError: name 'X' is not defined